In [1]:
# dobutamine :多巴酚丁胺
# dopamine :多巴胺
# epinephrine :肾上腺素
# norepinephrine:去甲肾上腺素
# phenylephrine :苯肾上腺素
# vasopressin : 血管加压素


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 300
import datetime as dt
import math
import seaborn as sns
import miceforest as mf
from tableone import TableOne, load_dataset

In [3]:
import datetime
def dateHourRange(beginDateHour, endDateHour):
    dhours = []
    dhour = beginDateHour #datetime.datetime.strptime(beginDateHour, "%Y-%m-%d %H:%M:%S")
    date = beginDateHour
    while date <= endDateHour:
        dhours.append(date)
        dhour = dhour + 1
        date = dhour
    return dhours

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# vaso

## 添加药物类型

In [4]:
# df_vaso = pd.read_csv("../../raw/treatment//vasopressors_inotropes.csv")
# vaso_dict = {'Dobutamine (Dobutrex)':"dobutamine", 'Dopamine (Inotropin)':"dopamine",
#        'Noradrenaline (Norepinefrine)':"norepinephrine", 'Adrenaline (Epinefrine)':"epinephrine"}
# df_vaso["drug"] = df_vaso["item"].map(vaso_dict)
# df_vaso.loc[:,['admissionid',"drug",'duration', 'rate', 'rateunit',
#        'start_time', 'stop_time', 'gamma']]
# df_vaso.to_csv("./vaso_clean.csv",index=False)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## to hourly

In [5]:
# df_vaso = pd.read_csv("./vaso_clean.csv")
# df_vaso = df_vaso.loc[:,['admissionid','start_time','stop_time',"gamma", "drug","duration"]]
# df_vaso.columns= ["stay_id", "starttime", "endtime", "vaso_amount", "vaso_type","duration_min"]
# df_vaso["starttime"] = df_vaso["starttime"]//60 # hour
# df_vaso["endtime"] = df_vaso["endtime"]//60 + 1 # hour
# df_vaso = df_vaso.loc[:,['stay_id','starttime','endtime',"vaso_amount", "vaso_type"]]
# df_vaso.head()

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# df_vaso["time_lst"] = df_vaso.apply(lambda x: dateHourRange(x['starttime'],x['endtime']),axis=1)
# df_vaso["time_num"] = df_vaso["time_lst"].map(lambda x: len(x))
# df_vaso["vaso_amount_hourly"] =  df_vaso["vaso_amount"]/df_vaso["time_num"] 

# df_vaso['idx'] = range(len(df_vaso))
# tem_add = df_vaso.set_index('idx').time_lst.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'charttime'})
# df_vaso_hourly = pd.merge(df_vaso, tem_add, how="inner", on="idx")
# df_vaso_hourly = df_vaso_hourly.loc[:,["stay_id","charttime","vaso_amount_hourly","vaso_type"]]
# df_vaso_hourly.to_csv("./vaso_hourly_with_type.csv",index=False)
# df_vaso_hourly.head(2)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# fuild

In [7]:
df_fluid = pd.read_csv("../../raw/treatment/fluid_colloid_and_crystalloid.csv")
df_fluid["during_hour"] =(df_fluid["endtime"]-df_fluid["starttime"])/60
df_fluid["rate"] = round(df_fluid["fluid"]/df_fluid["during_hour"])
df_fluid["rateunit"] = "ml/hour"

df_fluid = df_fluid.loc[:,['admissionid','starttime','endtime','fluid',"rate"]]
df_fluid.columns = ['stay_id','starttime','endtime', 'colloid_bolus',"rate"]
df_fluid["starttime"] = df_fluid["starttime"]//60 # hour
df_fluid["endtime"] = df_fluid["endtime"]//60 + 1 # hour
df_fluid = df_fluid.drop_duplicates()
df_fluid.to_csv("./aumc_fluidbanlance.csv",index=False)
df_fluid.head(2)


/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,starttime,endtime,colloid_bolus,rate
0,0,5,14,500.0,65.0
1,0,5,40,500.0,15.0


##  to hourly

In [8]:
df_fluid = pd.read_csv("./aumc_fluidbanlance.csv")

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,starttime,endtime,colloid_bolus,rate,time_lst,time_num,colloid_bolus_hourly
0,0,5,14,500.0,65.0,"[5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13...",10,50.000000
1,0,5,40,500.0,15.0,"[5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13...",36,13.888889


In [9]:
df_fluid["time_lst"] = df_fluid.apply(lambda x: dateHourRange(x['starttime'],x['endtime']),axis=1)
df_fluid["time_num"] = df_fluid["time_lst"].map(lambda x: len(x))
df_fluid["colloid_bolus_hourly"] =  df_fluid["colloid_bolus"]/df_fluid["time_num"] 

df_fluid['idx'] = range(len(df_fluid))
tem_add = df_fluid.set_index('idx').time_lst.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'charttime'})
df_fluid_hourly = pd.merge(df_fluid, tem_add, how="inner", on="idx")

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/pandas/core/series.py:4138: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [10]:
df_fluid_hourly = df_fluid_hourly[['stay_id', 'starttime', 'endtime', 'colloid_bolus', 'rate', 'colloid_bolus_hourly',  'charttime']]
df_fluid_hourly = df_fluid_hourly.rename(columns={"starttime":"fluid_starttime","endtime":"fluid_endtime"})
df_fluid_hourly.to_csv("./fluid_hourly.csv",index=False)
df_fluid_hourly.head(2)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,fluid_starttime,fluid_endtime,colloid_bolus,rate,colloid_bolus_hourly,charttime
0,0,5,14,500.0,65.0,50.0,5.0
1,0,5,14,500.0,65.0,50.0,6.0
